In [ ]:
!pip install einops

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 1.4 MB/s eta 0:00:00


In [ ]:
import torch.nn as nn
import torch
from einops import rearrange
import math
import torch.nn.functional as F
class ShiftedWindowMSA(nn.Module):

    def __init__(self, embed_dim, num_heads, window_size=7):
        super().__init__()
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.window_size = window_size
        self.proj1 = nn.Linear(embed_dim, 3*embed_dim)
        self.proj2 = nn.Linear(embed_dim, embed_dim)

    def forward(self, x):
        h_dim = self.embed_dim / self.num_heads
        height = width = int(math.sqrt(x.shape[1])) #sqrt of 3969
        x = self.proj1(x) #1,3969,288
        x = rearrange(x, 'b (h w) (c K) -> b h w c K', K=3, h=height, w=width) #1,63,63,96,3
        x = rearrange(x, 'b (h m1) (w m2) (H E) K -> b H h w (m1 m2) E K', H=self.num_heads, m1=self.window_size, m2=self.window_size)
        #1,3,7,7,81,32,3

        Q, K, V = x.chunk(3, dim=6)
        print(Q.shape)
        Q, K, V = Q.squeeze(-1), K.squeeze(-1), V.squeeze(-1)
        print(Q.shape)
        print(K.shape)
        print(K.transpose(4,5).shape)
        att_scores = (Q @ K.transpose(4,5)) / math.sqrt(h_dim)
        print(att_scores.shape)
        print(att_scores)
        att = F.softmax(att_scores, dim=-1) @ V

        x = rearrange(att, 'b H h w (m1 m2) E -> b (h m1) (w m2) (H E)', m1=self.window_size, m2=self.window_size)
        x = rearrange(x, 'b h w c -> b (h w) c')

        return self.proj2(x)

In [ ]:
test=torch.randn(1,3969,96)
obj=ShiftedWindowMSA(96,3)
out=obj(test)

torch.Size([1, 3, 9, 9, 49, 32, 1])
torch.Size([1, 3, 9, 9, 49, 32])
torch.Size([1, 3, 9, 9, 49, 32])
torch.Size([1, 3, 9, 9, 32, 49])
torch.Size([1, 3, 9, 9, 49, 49])
tensor([-0.0354,  0.4046, -0.2422, -0.0756,  0.3022,  0.3034,  0.0307,  0.1444,
        -0.0678, -0.7405,  0.5652, -0.2716, -0.0853, -0.0629,  0.0171,  0.5844,
         0.3814,  0.5636,  0.6189,  0.2669,  0.1224, -0.2086,  0.1773, -0.1738,
        -0.0245,  0.0737, -0.0845,  0.2991,  0.4986, -0.0886, -0.0094,  0.0422,
         0.0310, -0.4331, -0.1057,  0.2398,  0.3034,  0.2545,  0.1290, -0.2361,
        -0.3241,  0.6415, -0.1865,  0.0423, -1.0829, -0.1173,  0.1321, -0.2014,
         0.1646], grad_fn=<SliceBackward0>)
